In [17]:
import os
import torch
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from PIL import Image
from sklearn.model_selection import train_test_split
from google.colab import drive

In [18]:
# Use this to connect to Google Drive in Google Colab
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
# Use this to unzip file in Google Colab
!unzip drive/MyDrive/data

Archive:  drive/MyDrive/data.zip
replace __MACOSX/._data? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [22]:
class image_data_set(torch.utils.data.Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
  
    def __len__(self):
        return len(self.data)
  
    def __getitem__(self, index):
        return {'data': self.data[index], 'label': self.labels[index]}

In [23]:
def get_image_tensor(file_path):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    
    image = Image.open(file_path)
    return transform(image)

def get_data_and_labels(directory_path, label):
    image_tensors, labels = [], []
    for file in os.listdir(directory_path):
        if file.endswith(".JPG"):
            file_path = directory_path + file
            image_tensor = get_image_tensor(file_path)
            image_tensors.append(image_tensor)
            labels.append(label)
            
    return image_tensors, labels
    
def get_data_sets(): 
    present_file_path = "data/MP152_ODOT009_REPELCAM/present/"
    not_present_file_path = "data/MP152_ODOT009_REPELCAM/not_present/"
    
    present_data, present_labels = get_data_and_labels(present_file_path, 1)
    not_present_data, not_present_labels = get_data_and_labels(not_present_file_path, 0)
    
    print("Number of wildlife present photos: " + str(len(present_data)))
    print("Number of no wildlife present photos: " + str(len(not_present_data)))
    
    data = present_data + not_present_data
    labels = present_labels + not_present_labels
    
    training_data, testing_data, training_labels, testing_labels = train_test_split(data, labels)
    
    print("\nNumber of training photos: " + str(len(training_data)))
    print("Number of testing photos: " + str(len(testing_data)))
    
    training_data_set = image_data_set(training_data, training_labels)
    testing_data_set = image_data_set(testing_data, testing_labels)
    
    return training_data_set, testing_data_set

def get_loaders(training_data_set, testing_data_set, batch_size):
    training_loader = torch.utils.data.DataLoader(dataset = training_data_set,
                                                  batch_size = batch_size,
                                                  shuffle = True)

    testing_loader = torch.utils.data.DataLoader(dataset = testing_data_set,
                                                 batch_size = batch_size,
                                                 shuffle = True)
    
    return training_loader, testing_loader

In [25]:
def train(model, training_loader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    num_correct = 0
    for i, data in enumerate(training_loader):
        data, labels = data['data'].to(device), data['label'].to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, labels)
        running_loss += loss.item()
        _, predictions = torch.max(output.data, 1)
        num_correct += (predictions == labels).sum().item()
        loss.backward()
        optimizer.step()
    
    loss = running_loss/len(training_loader.dataset)
    accuracy = num_correct/len(training_loader.dataset)
    return loss, accuracy

def test(model, testing_loader, criterion):
    model.eval()
    running_loss = 0.0
    num_correct = 0
    for i, data in enumerate(testing_loader):
        data, labels = data['data'].to(device), data['label'].to(device)
        output = model(data)
        loss = criterion(output, labels)
        running_loss += loss.item()
        _, predictions = torch.max(output.data, 1)
        num_correct += (predictions == labels).sum().item()
    
    loss = running_loss/len(testing_loader.dataset)
    accuracy = num_correct/len(testing_loader.dataset)
    return loss, accuracy

In [26]:
def train_and_test(model, training_loader, testing_loader, device):
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    
    for epoch in range(5):
        print("epoch: " + str(epoch))
        
        training_loss, training_accuracy = train(model, training_loader, criterion, optimizer)
        print("training loss: " + str(training_loss) + " and training accuracy: " + str(training_accuracy))
        
        testing_loss, testing_accuracy = test(model, testing_loader, criterion)
        print("testing loss: " + str(testing_loss) + " and testing accuracy: " + str(testing_accuracy))

In [58]:
def train_and_test_VGG11(training_loader, testing_loader, device, num_classes):
    print("\nTraining and Testing VGG11")
    vgg11 = models.vgg11(weights = models.VGG11_Weights.DEFAULT)
    vgg11.classifier[6].out_features = num_classes
    train_and_test(vgg11, training_loader, testing_loader, device)

def train_and_test_VGG19(training_loader, testing_loader, device, num_classes):
    print("\nTraining and Testing VGG19")
    vgg19 = models.vgg19(weights = models.VGG19_Weights.DEFAULT)
    vgg19.classifier[6].out_features = num_classes
    train_and_test(vgg19, training_loader, testing_loader, device)

def train_and_test_ResNet50(training_loader, testing_loader, device, num_classes):
    print("\nTraining and Testing ResNet50")
    resnet50 = models.resnet50(weights = models.ResNet50_Weights.DEFAULT)
    resnet50.fc.out_features = num_classes
    train_and_test(resnet50, training_loader, testing_loader, device)

def train_and_test_ResNet152(training_loader, testing_loader, device, num_classes):
    print("\nTraining and Testing ResNet152")
    resnet152 = models.resnet152(weights = models.ResNet152_Weights.DEFAULT)
    resnet152.fc.out_features = num_classes
    train_and_test(resnet152, training_loader, testing_loader, device)
    
def train_and_test_AlexNet(training_loader, testing_loader, device, num_classes):
    print("\nTraining and Testing AlexNet")
    alexnet = models.alexnet(weights = models.AlexNet_Weights.DEFAULT)
    alexnet.classifier[6].out_features = num_classes
    train_and_test(alexnet, training_loader, testing_loader, device)

def train_and_test_SSD(training_loader, testing_loader, device, num_classes):
    print("\nTraining and Testing SSD")
    ssd300_vgg16 = models.detection.ssd300_vgg16(weights = models.detection.SSD300_VGG16_Weights.COCO_V1)
    train_and_test(ssd300_vgg16, training_loader, testing_loader, device)
    
def train_and_test_Faster_RCNN(training_loader, testing_loader, device, num_classes):
    print("\nTraining and Testing Faster R-CNN")
    faster_rcnn = models.detection.fasterrcnn_resnet50_fpn_v2(weights = models.detection.FasterRCNN_ResNet50_FPN_V2_Weights.DEFAULT)
    faster_rcnn.roi_heads.box_predictor.bbox_pred.out_features = num_classes
    train_and_test(faster_rcnn, training_loader, testing_loader, device)
    
def train_and_test_YOLO(training_loader, testing_loader, device, num_classes):
    print("\nTraining and Testing YOLO")
    yolo = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
    train_and_test(yolo, training_loader, testing_loader, device)

#Orchestration

In [29]:
num_classes = 2
batch_size = 10

training_data_set, testing_data_set = get_data_sets()
training_loader, testing_loader = get_loaders(training_data_set, testing_data_set, batch_size)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()

Number of wildlife present photos: 223
Number of no wildlife present photos: 363

Number of training photos: 439
Number of testing photos: 147


In [42]:
train_and_test_VGG11(training_loader, testing_loader, device, num_classes)


Training and Testing VGG11
epoch: 0
training loss: 0.13112048455142758 and training accuracy: 0.6993166287015945
testing loss: 0.05869498869188789 and testing accuracy: 0.7482993197278912
epoch: 1
training loss: 0.03983639561675133 and training accuracy: 0.806378132118451
testing loss: 0.033517539298453296 and testing accuracy: 0.9115646258503401
epoch: 2
training loss: 0.036134377605619085 and training accuracy: 0.8587699316628702
testing loss: 0.030418635377673066 and testing accuracy: 0.8843537414965986
epoch: 3
training loss: 0.027773917366268965 and training accuracy: 0.8838268792710706
testing loss: 0.02654149920559254 and testing accuracy: 0.9251700680272109
epoch: 4
training loss: 0.025258682099656798 and training accuracy: 0.9043280182232346
testing loss: 0.023901004572303926 and testing accuracy: 0.9183673469387755


In [43]:
train_and_test_VGG19(training_loader, testing_loader, device, num_classes)


Training and Testing VGG19
epoch: 0
training loss: 0.1013374178922801 and training accuracy: 0.7312072892938497
testing loss: 0.04370916259734809 and testing accuracy: 0.8639455782312925
epoch: 1
training loss: 0.041141403300140336 and training accuracy: 0.8200455580865603
testing loss: 0.03754961703504835 and testing accuracy: 0.8843537414965986
epoch: 2
training loss: 0.03266759247334509 and training accuracy: 0.8496583143507973
testing loss: 0.031066886624511406 and testing accuracy: 0.9115646258503401
epoch: 3
training loss: 0.03429192949960471 and training accuracy: 0.856492027334852
testing loss: 0.044656122420109856 and testing accuracy: 0.8231292517006803
epoch: 4
training loss: 0.025499521608301066 and training accuracy: 0.9020501138952164
testing loss: 0.021106130348480478 and testing accuracy: 0.9183673469387755


In [46]:
train_and_test_ResNet50(training_loader, testing_loader, device, num_classes)


Training and Testing ResNet50
epoch: 0
training loss: 0.3140658556328819 and training accuracy: 0.5375854214123007
testing loss: 0.06684430863480179 and testing accuracy: 0.891156462585034
epoch: 1
training loss: 0.04184125690532165 and training accuracy: 0.8838268792710706
testing loss: 0.038140732465552635 and testing accuracy: 0.9183673469387755
epoch: 2
training loss: 0.02655530384909319 and training accuracy: 0.9066059225512528
testing loss: 0.03527047913394818 and testing accuracy: 0.9251700680272109
epoch: 3
training loss: 0.022993019245276417 and training accuracy: 0.9157175398633257
testing loss: 0.028924452977216974 and testing accuracy: 0.9319727891156463
epoch: 4
training loss: 0.019036892046282124 and training accuracy: 0.9248291571753986
testing loss: 0.02365854581450524 and testing accuracy: 0.9387755102040817


In [50]:
train_and_test_ResNet152(training_loader, testing_loader, device, num_classes)


Training and Testing ResNet152
epoch: 0
training loss: 0.28502759133764716 and training accuracy: 0.5421412300683371
testing loss: 0.06027526811373477 and testing accuracy: 0.891156462585034
epoch: 1
training loss: 0.0359577101004015 and training accuracy: 0.8678815489749431
testing loss: 0.02974418202275727 and testing accuracy: 0.9251700680272109
epoch: 2
training loss: 0.023651025315284187 and training accuracy: 0.89749430523918
testing loss: 0.0232221309070279 and testing accuracy: 0.9115646258503401
epoch: 3
training loss: 0.018335462329327923 and training accuracy: 0.9384965831435079
testing loss: 0.02024666881378816 and testing accuracy: 0.9183673469387755
epoch: 4
training loss: 0.014728586374016178 and training accuracy: 0.9430523917995444
testing loss: 0.016580290281448234 and testing accuracy: 0.9523809523809523


In [51]:
train_and_test_AlexNet(training_loader, testing_loader, device, num_classes)


Training and Testing AlexNet
epoch: 0
training loss: 0.15341773968622732 and training accuracy: 0.6788154897494305
testing loss: 0.043597612149861395 and testing accuracy: 0.8775510204081632
epoch: 1
training loss: 0.03743935920349015 and training accuracy: 0.8587699316628702
testing loss: 0.04753332724477969 and testing accuracy: 0.8571428571428571
epoch: 2
training loss: 0.0325610366372014 and training accuracy: 0.8838268792710706
testing loss: 0.02820405450199737 and testing accuracy: 0.9183673469387755
epoch: 3
training loss: 0.02978592531666465 and training accuracy: 0.8815489749430524
testing loss: 0.029296493550547125 and testing accuracy: 0.8843537414965986
epoch: 4
training loss: 0.02730277657576737 and training accuracy: 0.9020501138952164
testing loss: 0.02709394561595657 and testing accuracy: 0.9183673469387755


In [56]:
#TODO: Get both of these working
#train_and_test_Faster_RCNN(training_loader, testing_loader, device, num_classes)
#train_and_test_YOLO(training_loader, testing_loader, device, num_classes)
#train_and_test_SSD(training_loader, testing_loader, device, num_classes)